In [ ]:
#50 ms 2*2 binning 100% power green channel ####preprocessing: fiji image->adjust->bleach correction-> simple ratio. 60pix rolling background subtraction. 60%~99.99%. ##### hitogram matching ; adjust contrast 67% to 99.9%

In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib as mpl
import matplotlib.pyplot as plt


# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import os

import pims
import trackpy as tp

from PIL import Image

from IPython import display
from scipy.optimize import curve_fit, least_squares

import tifffile
import diptest

In [ ]:
def r_squared(f,xdata,ydata,popt):

    residuals = ydata- f(xdata, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

In [ ]:
# set some environment and scaling variables
current_folder = os.getcwd()
folder = current_folder
output_folder = folder+'/micro_traces/' # output folder
#os.mkdir(output_folder)  # uncomment this line for creating the output dir

In [ ]:
experiments = [ '041822_dilution_concentrated_100nm']
experiment_folder = folder+'/'+experiments[0]+'/'

In [ ]:
os.listdir(experiment_folder)

In [ ]:
extract_files = []
for file in os.listdir(experiment_folder):
    if ('tif' in file) & ('_extract_a_100nm_' in file):
        extract_files.append(file)
extract_files.sort(key=lambda f: int(f.split('act_a_100nm_')[1].split('_RAW_ch00')[0]) )
conc_files = []
for file in os.listdir(experiment_folder):
    if ('tif' in file) & ('_concentrated_a_100nm_' in file):
        conc_files.append(file)
conc_files.sort(key=lambda f: int(f.split('concentrated_a_100nm_')[1].split('_RAW_ch00')[0]) )
files = extract_files+conc_files
for file in files:
    print(file)
#compile file for flatfield
tifffile_shape = tifffile.imread(experiment_folder+files[0]).shape

stack = tifffile.imread(experiment_folder+files[0], key=(0,tifffile_shape[0]-1 ,20))
for file in files[1:]:
    frames = tifffile.imread(experiment_folder+file, key=(0,tifffile_shape[0]-1 ,20))
    stack = np.concatenate((stack,frames))
tifffile.imwrite('for_flat_field.tif',stack)

In [ ]:
flatfield = pims.open(experiment_folder+'Flat_field.tif')

In [ ]:
f = tp.locate(np_frames[0], 7, invert=False, minmass = 250) #, minmass=50000)#130000)
f = tp.batch(np_frames[:300], 7, minmass=250);#50000);#130000, invert=False);
t = tp.link(f, 25, memory=3)
t1 = tp.filter_stubs(t, 10)
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

t2 = t1[((t1['mass'] > 100) & (t1['size'] < 5.0) )]
if len(t2) > 0:
    plt.figure(figsize = [10,10])
    tp.plot_traj(t2);
    plt.show()
    
plt.figure(figsize = [15,15])
#tp.annotate(f, frames[i])
plt.imshow(frames[0])
#plt.plot(f['x']+5,f['y']+5,'.')
plt.plot(f.loc[f['frame']==0]['x']+5,f.loc[f['frame']==0]['y'],'.')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=np.linspace(0,500,100) ) #(0,150000,100))

# Optionally, label the axes.
ax.set(xlabel='mass', ylabel='count');

# t = tp.link(f, 10, memory=3)
t1 = tp.filter_stubs(t, 10)
# Compare the number of particles in the unfiltered and filtered data.
#print('Before:', t['particle'].nunique())
#print('After:', t1['particle'].nunique())

t2 = t1[((t1['mass'] > 100) & (t1['size'] < 5.0) )]
plt.figure()
plt.xlim(0,2000)
tp.mass_size(t.groupby('particle').mean())
tp.mass_size(t.groupby('particle').mean())

In [ ]:
files

In [ ]:
def gray(image):
    return image[:, :]/2  # [:,:,1]Take just the green channel

fps = 308/86.038 #300/89.811 #367/110.249# 490/151.515
voxel = 0.322 
list_ns = []
list_As = []
list_ims = []
list_ems = []
list_ds = []
list_t2s = []
list_file_records = []


#for selected in [0]:#range(len(experiments)):
    #experiment_folder = current_folder+'/'+experiments[selected]
    #files = []
    #for file in os.listdir(experiment_folder):
    #    if 'tif' in file:
    #        files.append(file)
            
    #try:
    #    files.sort(key=lambda f: int(f.split('_A_')[1].split('_')[0]) )
    #except:
    #    print('aoe') #it used to be continue and skipped the ATP experiments
#for file in files: 
#    print(file)
tp.quiet()
ns = []
As = []
ims = []
ems = []
ds = []
t2s = []
file_records = []

#search_ranges = np.tile(np.linspace(25,15,11),2)
search_ranges = np.concatenate((np.array([20]),np.linspace(20,10,6),np.linspace(20,5,11))) #search_ranges = np.concatenate((np.array([25]),np.linspace(25,15,6),np.linspace(25,5,11)))
norm_flatfield = flatfield/np.mean(flatfield)

for i in range(len(files)):
    print(files[i])
    frames = pims.open(experiment_folder+'/'+files[i])
    overall_mean = np.mean(frames)
    np_frames = np.ones(np.shape(frames))
    for j,frame in enumerate(frames):
        norm_frame = frame*overall_mean/np.mean(frame)/norm_flatfield
        np_frames[j] = (norm_frame-np.percentile(np.array(norm_frame),50)).astype(int)
    np_frames = np_frames[0:150]
    
    diameter = 15
    maxsize = 7
    minmass =650
    percentile = 99.5
    topn = 300

    f = tp.locate(np_frames[0], diameter, invert=False, maxsize =maxsize, minmass = minmass, preprocess = True) #, minmass=50000)#130000)
    f = tp.batch(np_frames[:300], diameter, invert=False, maxsize =maxsize, minmass = minmass,topn=topn,preprocess = True)#diameter, minmass=250);#50000);#130000, invert=False);
    t = tp.link(f, search_ranges[i], memory=1,adaptive_stop=0.01, adaptive_step=0.98)
    t1 = tp.filter_stubs(t, 15)
    # Compare the number of particles in the unfiltered and filtered data.
    print('Before:', t['particle'].nunique())
    print('After:', t1['particle'].nunique())

    t2 = t1#[((t1['mass'] > 100) & (t1['size'] < 5.0) )]
    if len(t2) > 0:
        plt.figure(figsize = [10,10])
        tp.plot_traj(t2);
        plt.show()

    d = tp.compute_drift(t2)
    tm = tp.subtract_drift(t2.copy(), d)

    if len(tm) > 0:
        plt.figure(figsize = [10,10])
        tp.plot_traj(tm);
        plt.savefig(str(i)+'.pdf')
        plt.show()

        im = tp.imsd(tm, voxel, fps)#.head(15)  # microns per pixel = 100/285., frames per second = 24
        fig, ax = plt.subplots(figsize = [5,5])
        ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
        ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
               xlabel='lag time $t$')
        ax.set_xscale('log')
        ax.set_yscale('log')
        plt.show()

        em = tp.emsd(tm, voxel, fps).head(30) # microns per pixel = 100/285., frames per second = 24    
        plt.figure(figsize = [5,5])
        plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
        plt.xlabel('lag time $t$');
        para = tp.utils.fit_powerlaw(em)  # performs linear best fit in log space, plots]
        plt.show()
        print(files[i][0])
        print(para)
        ns.append(para['n'][0])
        As.append(para['A'][0])
    else:
        im = tm
        em = tm


    #tub_image = plt.imread('./'+categories[category-2]+'/Project004_Image '+str(get_image_num(files[i][0])-2)+'_RAW_ch00.tif')
    #tub_h,tub_w = np.shape(tub_image)
    #center_y, center_x = (int(tub_h/2),int(tub_w/2))
    #half_sq_h,half_sq_w = (int(tub_h/2/2/2/2),int(tub_w/2/2/2/2))
    #tub_center_median_intensities.append(np.median(tub_image[center_y-half_sq_h:center_y+half_sq_h,center_x-half_sq_w:center_x+half_sq_w].flatten())) # 40X -> 5X /2/2/2
    #tub_center_mean_intensities.append(np.mean(tub_image[center_y-half_sq_h:center_y+half_sq_h,center_x-half_sq_w:center_x+half_sq_w].flatten())) # 40X -> 5X /2/2/2
    #plt.imshow(tub_image)
    #plt.show()
    ims.append(im)
    ems.append(em)
    ds.append(d)
    t2s.append(t2)
    file_records.append(experiment_folder+'/'+files[i])

list_ns.append(ns)
list_As.append(As)
#list_tub_center_median_intensities.append(tub_center_median_intensities)
#list_tub_center_mean_intensities.append(tub_center_mean_intensities)
list_ims.append(ims)
list_ems.append(ems)
list_ds.append(ds)
list_t2s.append(t2s)
list_file_records.append(file_records)

In [ ]:
diameter = 15
maxsize = 7
minmass =1500
percentile = 99.5
topn = 300


f = tp.locate(np_frames[0], diameter, invert=False, maxsize =maxsize, minmass = minmass, preprocess = False) #, minmass=50000)#130000)


In [ ]:
plt.hist(f['mass'],bins=100);
plt.xlim(0,3000)
plt.show()

In [ ]:
for i in range(100):
    plt.figure(figsize = [15,15])
    plt.imshow(np_frames[i])
    tp.annotate(t2[t2['frame'] == i], frames[i]);
    plt.show()


In [ ]:
list_t2s

In [ ]:
def rand_cmap(nlabels, type='bright', first_color_black=True, last_color_black=False, verbose=True):
    """
    Creates a random colormap to be used together with matplotlib. Useful for segmentation tasks
    :param nlabels: Number of labels (size of colormap)
    :param type: 'bright' for strong colors, 'soft' for pastel colors
    :param first_color_black: Option to use first color as black, True or False
    :param last_color_black: Option to use last color as black, True or False
    :param verbose: Prints the number of labels and shows the colormap. True or False
    :return: colormap for matplotlib
    """
    from matplotlib.colors import LinearSegmentedColormap
    import colorsys
    import numpy as np


    if type not in ('bright', 'soft'):
        print ('Please choose "bright" or "soft" for type')
        return

    if verbose:
        print('Number of labels: ' + str(nlabels))

    # Generate color map for bright colors, based on hsv
    if type == 'bright':
        randHSVcolors = [(np.random.uniform(low=0.0, high=1),
                          np.random.uniform(low=0.2, high=1),
                          np.random.uniform(low=0.9, high=1)) for i in range(nlabels)]

        # Convert HSV list to RGB
        randRGBcolors = []
        for HSVcolor in randHSVcolors:
            randRGBcolors.append(colorsys.hsv_to_rgb(HSVcolor[0], HSVcolor[1], HSVcolor[2]))

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]

        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Generate soft pastel colors, by limiting the RGB spectrum
    if type == 'soft':
        low = 0.6
        high = 0.95
        randRGBcolors = [(np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high)) for i in range(nlabels)]

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]
        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Display colorbar
    if verbose:
        from matplotlib import colors, colorbar
        from matplotlib import pyplot as plt
        fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))

        bounds = np.linspace(0, nlabels, nlabels + 1)
        norm = colors.BoundaryNorm(bounds, nlabels)

        cb = colorbar.ColorbarBase(ax, cmap=random_colormap, norm=norm, spacing='proportional', ticks=None,
                                   boundaries=bounds, format='%1i', orientation=u'horizontal')

    return random_colormap

In [ ]:
rand_cmap(np.max(f['particle']), type='bright', first_color_black=True, last_color_black=False, verbose=True)

In [ ]:
plt.scatter(to_plot['x']+10,to_plot['y'],marker = '.', c = colors,  cmap=new_cmap)

In [ ]:
k = 7
f = list_t2s[0][k]
frames = pims.open(experiment_folder+'/'+files[k])
overall_mean = np.mean(frames)
np_frames = np.ones(np.shape(frames))
for j,frame in enumerate(frames):
    norm_frame = frame*overall_mean/np.mean(frame)/norm_flatfield
    np_frames[j] = (norm_frame-np.percentile(np.array(norm_frame),50)).astype(int)
new_cmap = rand_cmap(np.max(f['particle']), type='bright', first_color_black=True, last_color_black=False, verbose=True)
for i in range(100):
    plt.figure(figsize = [10,10])
    #f = tp.locate(frames[i], 15, invert=False, minmass=1500)
    #tp.annotate(f, frames[i])
    to_plot = f.loc[f['frame']==i]
    colors = to_plot['particle']
    plt.imshow(np_frames[i])
    plt.scatter(to_plot['x']+10,to_plot['y'],marker = '.', c = colors, cmap=new_cmap, vmin=0, vmax=np.max(f['particle']))
    plt.xlim(0,1024)
    plt.show()
    display.clear_output(wait=True)

In [ ]:
import pickle

# obj0, obj1, obj2 are created here...
 
# Saving the objects:
#with open('050622_041822_objs1.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#    pickle.dump([list_ns, list_As, list_ims,list_ems,list_ds,list_t2s,list_file_records], f)

# Getting back the objects:
#with open('041822_objs1.pkl','rb') as f:  # Python 3: open(..., 'rb')
#    list_ns, list_As, list_ims,list_ems,list_ds,list_t2s,list_file_records = pickle.load(f)

In [ ]:
len(['extract_00']+['extr'+str(x*20) for x in range(0,6)]+['conc'+str(x*20) for x in range(0,11)])

In [ ]:
def normal_diffusion(x, D):
    return D * x 
colors = plt.cm.viridis(np.linspace(0,1,11))
Ds_list = []
for i in range(len(list_ems[0])):
    Ds = []
    xdata = list_ims[0][i].index
    for j in range(len(list_ims[0][i].columns)):
        ydata = [x/4 for x in list_ims[0][i].iloc[:,j]]
        popt, pcov = curve_fit(normal_diffusion, xdata[:3], ydata[:3])
        plt.scatter(xdata,ydata,color = colors[i%11],marker= 'o')
        plt.plot(xdata,normal_diffusion(xdata,popt[0]),color = colors[i%11])
        plt.ylim(0,10)
        Ds.append(popt[0])
    Ds_list.append(Ds)

In [ ]:
def cv(x):
    return np.std(x) / np.mean(x)
colors = plt.cm.jet(np.linspace(0,1,6))
plt.figure(figsize = [5,5])
bins = np.linspace(0,8,100)
for i,Ds in enumerate(Ds_list[1:7]):
    a = np.histogram(Ds,bins = bins,density=True);
    plt.step(bins[:-1],a[0]+i,color = 'black',alpha = i/7+1/7)
    #plt.ylim(0,8)
    print(cv(Ds))
    dip, pval = diptest.diptest(np.array(Ds))
    print((dip,pval))
plt.xlabel(r'$D_{eff}$ ($\mu m^2$/s)')
plt.savefig('modality.pdf')